In [12]:
from scipy.stats import norm
import numpy as np
import yfinance as yf
import pandas as pd

from datetime import datetime


def get_historical_data(stk, start_date, end_date):
    df = yf.download(stk, start_date, end_date, interval="1d").dropna()
    return df


def group_by_time_interval(df, number_of_weeks_in_group):
    df["start"] = df.index
    df["end"] = df.index
    df["day_of_week"] = df.index.dayofweek
    df["week_of_year"] = df.index.isocalendar().week + df.index.isocalendar().year * 100
    df["month_of_year"] = df.index.strftime("%m-%Y")
    df["week_group"] = (df.week_of_year / number_of_weeks_in_group).apply(np.ceil)
    df = df.groupby(["week_group"]).agg(
        {"start": "first", "end": "last", "Open": "first", "Close": "last", "Volume":"sum"}
    )
    #df['volume_rank']=df.Volume.rank(pct=True).shift(1)

    df["days"] = (df["end"] - df["start"]).dt.days + 1
    df = df.set_index("start")
    return df


def trade_parameters(df, yield_perc):
    df["strike_price"] = df.Open.round(0)
    df["straddle_yield"] = yield_perc
    df["straddle_premium"] = df.strike_price * yield_perc
    df['abs_change_pct']=(abs((df.Close - df.Open)/df.Open))
    df["price_change_abs"] = abs(df.Close - df.strike_price)
    df["profit_loss"] = df.straddle_premium - df.price_change_abs
    df["log_return"] = np.log((df.Open + df.profit_loss) / df.Open)
    df['volume_rank']=df.Volume.rank(pct=True)
    df['change_rank']=df.abs_change_pct.rank(pct=True)
    df['volume_rank1']=df.Volume.rank(pct=True).shift(1)
    df['change_rank1']=df.abs_change_pct.rank(pct=True).shift(1)

    df = df.iloc[1: , :]
    
    df=df[df.volume_rank1<0.80]
    df=df[df.change_rank1<0.80]
    return df





start_date = "2016-08-01"
end_date = "2022-08-13"
number_of_weeks_in_group = 2
arr = []


stock ="XRT"
symbols=('HYG','MSOS','GDX','XRT')
yields=(1.89,11.57,6.07,5.68)


for x,y in zip(symbols,yields):
    stock =x
    yield_perc = y/100
    df = get_historical_data(stock, start_date, end_date)
    df = group_by_time_interval(df, number_of_weeks_in_group)
    df = trade_parameters(df, yield_perc)
    df.to_csv(stock + ".csv")
    log_r = np.sum(df.log_return)
    pos_log = np.sum(df[df.log_return > 0].log_return)
    neg_log = np.sum(df[df.log_return <= 0].log_return)
    pos_periods = len(df[df.log_return > 0].log_return)
    ttl_periods = len(df.log_return)
    avg_yield = np.average(df.log_return.dropna())
    vol_rank=df.volume_rank.iloc[-1]
    lr_rank=df.change_rank.iloc[-1]
    max_draw_down=np.min(df.log_return)
    std=np.std(df.abs_change_pct)
    arr.append(
        [
            stock,
            yield_perc,
            log_r,
            pos_log,
            neg_log,
            pos_periods,
            ttl_periods,
            avg_yield,
            vol_rank,
            lr_rank,
            max_draw_down,
            std
        ]
    )

finaldf = pd.DataFrame(
    data=arr,
    columns=[
        "stock",
        "yield_perc",
        "log_r",
        "pos_log",
        "neg_log",
        "pos_periods",
        "ttl_periods",
        "avg_yield",
        'vol_rank',
        'lr_rank',
        'max_draw',
        'std'

    ],
)
finaldf["sharpe"] = abs(finaldf.pos_log / finaldf.neg_log)
finaldf["winr"] = finaldf.pos_periods / finaldf.ttl_periods
#finaldf.sort_values(by="log_r", ascending=False, inplace=True)
finaldf.to_csv("big.csv")
print(finaldf)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
  stock  yield_perc     log_r   pos_log   neg_log  pos_periods  ttl_periods  \
0   HYG      0.0189  1.278490  1.330120 -0.051630          106          112   
1  MSOS      0.1157  1.265403  1.739115 -0.473712           28           35   
2   GDX      0.0607  2.017168  2.619317 -0.602149           82          107   
3   XRT      0.0568  2.288402  2.871139 -0.582737           92          107   

   avg_yield  vol_rank   lr_rank  max_draw       std     sharpe      winr  
0   0.011415  0.905063  0.670886 -0.015952  0.006323  25.762669  0.946429  
1   0.036154  0.519231  0.288462 -0.122217  0.058786   3.671247  0.800000  
2   0.018852  0.177215  0.443038 -0.067124  0.028192   4.349947  0.766355  
3   0.021387  0.506329  